## Task 1

In [ ]:
! pip install transformers datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

ds1 = load_dataset("KaungHtetCho/Harry_Potter_LSTM")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import TfidfVectorizer

torch.manual_seed(100)

sample_sentence = sample_sentence = ds1['test'][29]['text']

char_set = list(set(sample_sentence))
dic = {c: i for i, c in enumerate(char_set)}

dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size * 2
output_size = dic_size
unit_sequence_length = 20

input_batch = []
target_batch = []

for i in range(len(sample_sentence) - unit_sequence_length):
    x_data = [dic[c] for c in sample_sentence[i:i + unit_sequence_length]]
    x_one_hot = [np.eye(len(dic))[x] for x in x_data]
    y_data = [dic[c] for c in sample_sentence[i + 1:i + unit_sequence_length + 1]]

    input_batch.append(x_one_hot)
    target_batch.append(y_data)

X = torch.FloatTensor(np.array(input_batch))
Y = torch.LongTensor(np.array(target_batch))
print(X.shape)
print(Y.shape)

class Custom_RNN(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, output_dim, layers):
    super(Custom_RNN, self).__init__()
    self.rnn = torch.nn.GRU(input_dim, hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(hidden_dim, output_dim, bias=True)

  def forward(self, x):
    x, _status = self.rnn(x)
    x = self.fc(x)
    return x

learning_rate = 0.05
training_epochs = 100
model = Custom_RNN(input_size, hidden_size, output_size, 2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(training_epochs):
  optimizer.zero_grad()
  outputs = model(X)
  loss = criterion(outputs.reshape(-1, dic_size), Y.reshape(-1))
  loss.backward()
  optimizer.step()
  if epoch % 10 == 9:
    print('epoch: ',epoch, 'loss: ', loss.item())

results = outputs.data.numpy().argmax(axis=2)

predicted_sentence = ""
predicted_sentence += ''.join([char_set[c] for c in np.squeeze(results[0])])
for i in range(1, len(results)):
    predicted_sentence += char_set[results[i][unit_sequence_length - 1]]

print(predicted_sentence)

vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform([sample_sentence, predicted_sentence])

cosine_sim = (tfidf_matrix * tfidf_matrix.T).toarray()[0, 1]

print("Accurracy = ", cosine_sim)

torch.Size([109, 20, 25])
torch.Size([109, 20])
epoch:  9 loss:  2.5646939277648926
epoch:  19 loss:  1.2510313987731934
epoch:  29 loss:  0.3829200267791748
epoch:  39 loss:  0.171338751912117
epoch:  49 loss:  0.12153716385364532
epoch:  59 loss:  0.10563018172979355
epoch:  69 loss:  0.09938982874155045
epoch:  79 loss:  0.09684986621141434
epoch:  89 loss:  0.09558268636465073
epoch:  99 loss:  0.09485244750976562
oldemort did not laugh. His gaze had wandered upward to the body revolving slowly overhead, and he seemed to be lost in thought.
Accurracy =  0.9239535357171462


## Task 2

In [ ]:
from datasets import load_dataset

ds2 = load_dataset("sepidmnorozy/Korean_sentiment")

In [ ]:
import evaluate

accuracy = evaluate.load('accuracy')

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

In [ ]:
# Model 1
tokenizer = AutoTokenizer.from_pretrained("WhitePeak/bert-base-cased-Korean-sentiment")
model_1 = AutoModelForSequenceClassification.from_pretrained("WhitePeak/bert-base-cased-Korean-sentiment")
classifier_1 = pipeline('sentiment-analysis', model="WhitePeak/bert-base-cased-Korean-sentiment", device=0)

In [ ]:
# Model 2
tokenizer = AutoTokenizer.from_pretrained("matthewburke/korean_sentiment")
model_2 = AutoModelForSequenceClassification.from_pretrained("matthewburke/korean_sentiment")
classifier_2 = pipeline("text-classification", model="matthewburke/korean_sentiment", device=0)

In [ ]:
test_list = ds2['test']
test_sample_size = 100
rng = np.random.default_rng()
random_indices = rng.choice(len(test_list), size=test_sample_size, replace=False)
sampled_data = test_list.select(random_indices)

test_predictions_1 = classifier_1(sampled_data['text'], max_length=512, truncation=True)
predicted_labels_1 = [label['label'] for label in test_predictions_1]
mapped_labels_1 = [model.config.label2id[label] for label in predicted_labels_1]
reference_labels_1 = sampled_data['label']
print("Model 1")
print(accuracy.compute(predictions=mapped_labels_1, references=reference_labels_1))
print(reference_labels_1)
print(mapped_labels_1)

test_predictions_2 = classifier_2(sampled_data['text'], max_length=512, truncation=True)
predicted_labels_2 = [label['label'] for label in test_predictions_2]
mapped_labels_2 = [model.config.label2id[label] for label in predicted_labels_2]
reference_labels_2 = sampled_data['label']
print("Model 2")
print(accuracy.compute(predictions=mapped_labels_2, references=reference_labels_2))
print(reference_labels_2)
print(mapped_labels_2)

Model 1
{'accuracy': 0.76}
[0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]
[0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0]
Model 2
{'accuracy': 0.89}
[0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0]
[0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 